# Data manipulation

Read the full COMPAS data, and extract only the information that you need (specifically only the systems that are every a DCO)

Then we want to add relevant information about the system in the following cases:

 * The first mass transfer that the binary engaged in
 * The first mass transfer that star 2 engaged in
 * The mass transfer that lead to a stellar merger (if any)
 * The supernova information

In [22]:
import numpy as np
import os 
import pandas as pd
import h5py as h5
import matplotlib.pyplot as plt
import gc

home_dir = os.path.expanduser("~") 
compas_v = "v03.01.02" #"v02.46.01/"#v02.35.02/"
datar_root =  f"{home_dir}/ceph/CompasOutput/{compas_v}/"

sim_name =  'NewWinds_RemFryer2012_noMSwinds'#'OldWinds_RemFryer2012'#  

######################################
## PLOT setttings
plt.rc('font', family='serif')
from matplotlib import rc
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
fsize, SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE = 30,20,25,30
for obj in ['axes','xtick','ytick']:
    plt.rc(obj, labelsize=SMALL_SIZE)          # controls default text sizes
for obj in ['figure','axes']:
    plt.rc(obj, titlesize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize


# Turn off natural name warning for panda tables (this is due to '@' and '>' in the COMPAS column names)
import warnings
from tables import NaturalNameWarning
warnings.filterwarnings('ignore', category=NaturalNameWarning)

## Get the potential DCO progenitors

Read the full COMPAS data, and extract systems that become a DCO at any of the simulated metallicities

In [23]:
save_name_table = 'potential_DCO_progenitors.h5'

# Initialize a list to store all SEEDS that ever become a DCO
All_DCO_seeds = []

# check if your table exists
if os.path.isfile(datar_root+ f'/{sim_name}/{save_name_table}'):
    print('Table already exists, loading it')
    potential_DCO_progenitors = pd.read_hdf(datar_root + f'/{sim_name}/{save_name_table}', key='All_DCO')

else:
    # Loop over all directories starting wiht "logZ"
    print(f'reading from {datar_root}/{sim_name}/')
    for i, dir in enumerate(os.listdir(datar_root+ f'/{sim_name}/')):

        if dir.startswith('logZ'):
            print(f"Reading DCO seeds from {dir}")

            # Open the HDF5 file for all systems at a given metallicity
            data = h5.File(datar_root+ f'/{sim_name}/{dir}/COMPAS_Output.h5', 'r')

            # Get the seeds that ever become a DCO
            DCO_seeds = pd.Series(data['BSE_Double_Compact_Objects']['SEED'][()])

            # Add them to the list of all DCO seeds
            All_DCO_seeds.extend(DCO_seeds)
        else:
            continue

    # take the unique seeds (some SEEDS might make a DCO at multiple metallicities)
    All_DCO_seeds  = np.unique(All_DCO_seeds)
    print('All_DCO_seeds', All_DCO_seeds, len(All_DCO_seeds) )

    # Open the HDF5 file for all systems at all metallicities (This is heavy on the memory)
    All_data = h5.File(datar_root+ f'/{sim_name}/COMPAS_Output_combinedZ.h5', 'r')

    # Create a mask to select only the systems that could potentially become a DCO
    SYS_mask = np.in1d(All_data['BSE_System_Parameters']['SEED'][()], All_DCO_seeds)

    # Read the HDF5 datasets as pandas dataframes
    SYS = pd.DataFrame()
    # chosen to allow for rerunning of systems and other interesting parameters
    SYS_keys_of_interest = ['SEED', 'Metallicity@ZAMS(1)', 'Stellar_Type(1)', 'Stellar_Type(2)','CE_Event_Counter', 'Mass@ZAMS(1)', 'Mass@ZAMS(2)','SemiMajorAxis@ZAMS',
                            'Merger','Merger_At_Birth','Unbound', 'Immediate_RLOF>CE','Optimistic_CE', 'Applied_Kick_Magnitude(1)', 'Applied_Kick_Magnitude(2)', 'CH_on_MS(1)',
                            'SN_Kick_Magnitude_Random_Number(1)','SN_Kick_Phi(1)','SN_Kick_Theta(1)','SN_Kick_Mean_Anomaly(1)',
                            'SN_Kick_Magnitude_Random_Number(2)','SN_Kick_Phi(2)','SN_Kick_Theta(2)','SN_Kick_Mean_Anomaly(2)' ]
    for key in SYS_keys_of_interest:
        # You cant directly apply the mask to the HDF5 dataset, so you have to read it first
        read_data = All_data['BSE_System_Parameters'][key][()]
        SYS[key] = read_data[SYS_mask]

    # Same mask for the DCO
    DCO_mask = np.in1d(All_data['BSE_Double_Compact_Objects']['SEED'][()], All_DCO_seeds)

    DCO = pd.DataFrame()
    DCO_keys_of_interest = ['SEED', 'Metallicity@ZAMS(1)', 'Merges_Hubble_Time', 'SemiMajorAxis@DCO','Coalescence_Time', 'Eccentricity@DCO', 'MT_Donor_Hist(1)', 'MT_Donor_Hist(2)', 'Mass(1)', 'Mass(2)']
    for key in DCO_keys_of_interest:
        read_data = All_data['BSE_Double_Compact_Objects'][key][()]
        DCO[key] = read_data[DCO_mask]

    # Merge the SYS and DCO dataframes to make potential_DCO_progenitors
    potential_DCO_progenitors = SYS.merge(DCO, on=['SEED', 'Metallicity@ZAMS(1)'], how='left')
    # Create a unique SEED that is a combination of SEED and metallicity
    potential_DCO_progenitors['unique_Z_SEED'] = [f"{seed}_{Z:.5f}" for seed, Z in zip(potential_DCO_progenitors['SEED'], potential_DCO_progenitors['Metallicity@ZAMS(1)'])]

    # test that this worked (every seed should occur 7 times, once for each Z)
    potential_DCO_seeds, counts = np.unique(potential_DCO_progenitors['SEED'], return_counts=True)
    print('potential_DCO_seeds', potential_DCO_seeds, 'counts', counts)

    # Save the total dataframe
    potential_DCO_progenitors.to_hdf(datar_root+ f'/{sim_name}/{save_name_table}', key='All_DCO', mode='w')




reading from /mnt/home/lvanson/ceph/CompasOutput/v03.01.02//NewWinds_RemFryer2012_noMSwinds/
Reading DCO seeds from logZ-1.85
Reading DCO seeds from logZ-3.26
Reading DCO seeds from logZ-1.52
Reading DCO seeds from logZ-3.0
Reading DCO seeds from logZ-2.4
Reading DCO seeds from logZ-1.7
Reading DCO seeds from logZ-2.7
Reading DCO seeds from logZ-3.52
Reading DCO seeds from logZ-3.76
Reading DCO seeds from logZ-2.0
Reading DCO seeds from logZ-2.2
Reading DCO seeds from logZ-4.0
All_DCO_seeds [      3       7       9 ... 4999991 4999994 4999996] 662872
potential_DCO_seeds [      3       7       9 ... 4999991 4999994 4999996] counts [12 12 12 ... 12 12 12]


/tmp/ipykernel_1403065/1037347188.py:71: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['MT_Donor_Hist(1)', 'MT_Donor_Hist(2)', 'unique_Z_SEED'], dtype='object')]

  potential_DCO_progenitors.to_hdf(datar_root+ f'/{sim_name}/{save_name_table}', key='All_DCO', mode='w')


In [24]:
display(potential_DCO_progenitors)

,SEED,Metallicity@ZAMS(1),Stellar_Type(1),Stellar_Type(2),CE_Event_Counter,Mass@ZAMS(1),Mass@ZAMS(2),SemiMajorAxis@ZAMS,Merger,Merger_At_Birth,...,SN_Kick_Mean_Anomaly(2),Merges_Hubble_Time,SemiMajorAxis@DCO,Coalescence_Time,Eccentricity@DCO,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),unique_Z_SEED
0,3600004,0.014142,14,14,0,60.628779,36.346818,1.795626,0,0,...,5.827147,0.0,2.333744,2.871559e+09,0.108141,b'2 ',b'2 ',14.448016,9.292275,3600004_0.01414
1,3600017,0.014142,14,2,1,28.199632,25.659874,2.087447,1,0,...,1.269687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3600017_0.01414
2,3600032,0.014142,13,13,1,22.159122,12.036042,1.450138,0,0,...,5.467253,1.0,0.016050,2.337468e+03,0.000000,b'2 ',b'4-8 ',2.109324,1.277584,3600032_0.01414
3,3600043,0.014142,14,14,0,69.679098,41.052767,14.947039,0,0,...,4.014026,0.0,62.232675,4.361167e+14,0.002159,b'2 ',b'NA ',16.752780,18.646141,3600043_0.01414
4,3600048,0.014142,14,13,0,54.888392,14.406964,0.414274,0,0,...,4.294370,0.0,0.747668,3.641520e+08,0.215558,b'2 ',b'2 ',11.740214,1.502387,3600048_0.01414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7954459,3849974,0.000100,7,1,1,30.878397,22.091519,2.246381,1,0,...,0.117612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3849974_0.00010
7954460,3849978,0.000100,14,13,0,24.342615,11.257121,150.325754,0,0,...,1.498153,0.0,480.352454,1.631730e+19,0.542207,b'NA ',b'NA ',14.435844,1.361065,3849978_0.00010
7954461,3849980,0.000100,14,13,0,20.528723,8.785796,177.199392,0,0,...,2.288087,0.0,586.404606,1.679910e+20,0.396746,b'NA ',b'NA ',9.244544,1.277584,3849980_0.00010
7954462,3849985,0.000100,7,1,1,16.023803,9.002761,2.055993,1,0,...,3.725517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3849985_0.00010


### Now we add the RLOF information to the potential DCO progenitor table

Use the above created table to start from and add
* mass transfer 1
* First MT from star 2
* the mass transfer that lead to a merger

### And in the end also supernova information 

#####  'Supernova_State'
*  No supernova = 0 
*  Star 1 is the supernova 	 = 1 
*  Star 2 is the supernova 	 = 2 
*  Both stars are supernovae 	 = 3

##### 'SN_Type(SN)'
*  NONE 	 = 0 
*  CCSN 	 = 1 
*  ECSN 	 = 2 
*  PISN 	 = 4 
*  PPISN 	 = 8 
*  USSN 	 = 16 
*  AIC 	     = 32 
*  SNIA 	 = 64 
*  HeSD 	 = 128

In [25]:
save_name_table = 'potential_DCO_progenitors_RLOFinfo.h5'

# check if your table exists
if os.path.isfile(datar_root+ f'/{sim_name}/{save_name_table}'):
    print('Table already exists, loading it')
    potential_DCO_progenitors = pd.read_hdf(datar_root + f'/{sim_name}/{save_name_table}', key='All_DCO')

else:
    # Read the beginning of the potential DCO progenitors table that you made above
    potential_DCO_progenitors = pd.read_hdf(f'{datar_root}/{sim_name}/potential_DCO_progenitors.h5', key='All_DCO')

    # Initialize a list to store all SEEDS that ever become a DCO
    All_DCO_seeds = []

    # Loop over all directories starting wiht "logZ"
    for i, dir in enumerate(os.listdir(datar_root+ f'/{sim_name}/')):
        # Get the seeds that ever become a DCO
        if dir.startswith('logZ'):
            print(f"Reading DCO seeds from {dir}") 
            data = h5.File(datar_root+ f'/{sim_name}/{dir}/COMPAS_Output.h5', 'r')
            DCO_seeds = pd.Series(data['BSE_Double_Compact_Objects']['SEED'][()])
            All_DCO_seeds.extend(DCO_seeds)
            
        else:
            continue
        

    # take the unique seeds (some SEEDS might make a DCO at multiple metallicities)
    All_DCO_seeds  = np.unique(All_DCO_seeds)

    # Open the HDF5 file for all systems at a given metallicity
    with h5.File(datar_root+ f'/{sim_name}/COMPAS_Output_combinedZ.h5', 'r') as All_data:
        ####################################
        # Read RLOF data
        RLOF = pd.DataFrame()
        # Select only the RLOF events for systems that could potentially become a DCO
        RLOF_mask = np.in1d(All_data['BSE_RLOF']['SEED'][()], All_DCO_seeds)

        ########################################################################################
        # Create the new RLOF columns in the potential DCO progenitors table
        print('start reading RLOF data')
        RLOF_keys = ['SEED', 'Metallicity@ZAMS(1)','SemiMajorAxis<MT', 'SemiMajorAxis>MT', 'Radius(1)<MT', 'Radius(2)<MT', 'Radius(1)>MT', 
                    'Radius(2)>MT', 'Mass(1)<MT', 'Mass(2)<MT', 'Mass(1)>MT', 'Mass(2)>MT','Stellar_Type(1)<MT', 'Stellar_Type(2)<MT', 
                    'Stellar_Type(1)>MT', 'Stellar_Type(2)>MT', 'MT_Event_Counter', 'CEE>MT', 'RLOF(1)>MT', 'RLOF(2)>MT', 'Merger']
        for key in RLOF_keys:
            read_data = All_data['BSE_RLOF'][key][()]
            RLOF[key] = read_data[RLOF_mask]
        print('add a few extra cols to RLOF')
        RLOF['unique_Z_SEED'] = [f"{seed}_{Z:.5f}" for seed, Z in zip(RLOF['SEED'], RLOF['Metallicity@ZAMS(1)'])]
        RLOF['M1_M2<MT'] = RLOF['Mass(1)<MT']/RLOF['Mass(2)<MT'].astype(float)
        RLOF.rename(columns={'Merger': 'RLOF_Merger'}, inplace=True) # rename so it wont conflict with the SYS['Merger']


    ########################################################################################
    # Now make subselections of this RLOF table that we can later add to the potential DCO progenitors table
    # first_RLOF_table    = first_RLOF_table.add_prefix('firstMT_')                                               # Add prefix to all the keys
    # first_RLOF_table.rename(columns={'firstMT_unique_Z_SEED': 'unique_Z_SEED'}, inplace=True)                   # Except the unique_Z_SEED
    ################################
    # First MT event
    print('Adding the MT information for the first MT ')
    first_MT_event_bool = RLOF['MT_Event_Counter'] == 1
    first_RLOF_table    = RLOF[first_MT_event_bool]

    # Add prefix to all the keys, Except 'SEED', 'Metallicity@ZAMS(1)', 'unique_Z_SEED'
    first_RLOF_table.rename(columns={col: 'firstMT_' + col for col in first_RLOF_table.columns if col not in ['SEED', 'Metallicity@ZAMS(1)', 'unique_Z_SEED']}, inplace=True)

    # a 'first MT' should only happen once per unique_Z_SEED
    s, counts = np.unique(first_RLOF_table['unique_Z_SEED'], return_counts = True)
    print('this should be 1', np.unique(counts))

    ################################
    # mass transfer that lead to a merger
    print('Adding the MT information for the mass transfer that lead to a merger')
    RLOF_Merger_bool            = RLOF['RLOF_Merger'] == 1
    MT_leading_to_merger_table  = RLOF[RLOF_Merger_bool]

    # Add prefix to all the keys, Except 'SEED', 'Metallicity@ZAMS(1)', 'unique_Z_SEED'
    MT_leading_to_merger_table.rename(columns={col: 'MT_lead_to_merger_' + col for col in MT_leading_to_merger_table.columns if col not in ['SEED', 'Metallicity@ZAMS(1)', 'unique_Z_SEED']}, inplace=True)

    # a 'mass transfer leading to stellar merger' should only happen once per unique_Z_SEED
    s, counts = np.unique(MT_leading_to_merger_table['unique_Z_SEED'], return_counts = True)
    print('this should be 1', np.unique(counts))

    ################################################################
    # First mass transfer from the second star
    print('Adding the MT information for the first MT from star 2')
    star_2_is_RLOF              = RLOF['RLOF(2)>MT'] == 1 # Star 2 is RLOF
    star_2_is_RLOF_table        = RLOF[star_2_is_RLOF]

    # Find the minimum 'MT_Event_Counter' for each 'unique_Z_SEED' where Star 2 is RLOF
    Minimun_MT_event_count_bool = np.where(star_2_is_RLOF_table['MT_Event_Counter'] == star_2_is_RLOF_table.groupby('unique_Z_SEED')['MT_Event_Counter'].transform('min'), True, False)
    first_MT_from_star2_table   = star_2_is_RLOF_table[Minimun_MT_event_count_bool].copy()

    # Add prefix to all the keys, Except 'SEED', 'Metallicity@ZAMS(1)', 'unique_Z_SEED'
    first_MT_from_star2_table.rename(columns={col: 'star2_firstMT_' + col for col in first_MT_from_star2_table.columns if col not in ['SEED', 'Metallicity@ZAMS(1)', 'unique_Z_SEED']}, inplace=True)

    # lastly, the first MT from star 2 should also only happen once per unique_Z_SEED
    s, counts = np.unique(first_MT_from_star2_table['unique_Z_SEED'], return_counts = True)
    print('this should be 1', np.unique(counts))

    ########################
    # Empty RLOF to save mem
    del RLOF
    gc.collect()  # Force garbage collector to release unreferenced memory

    print('start merging tables')
    # Merge this info with the potential_DCO_progenitors
    potential_DCO_progenitors = potential_DCO_progenitors.merge(first_RLOF_table, on=['SEED', 'Metallicity@ZAMS(1)', 'unique_Z_SEED'], how='left')
    print('done with first_RLOF_table')
    del first_RLOF_table # to save memory
    # Merge info with the potential_DCO_progenitors
    potential_DCO_progenitors = potential_DCO_progenitors.merge(MT_leading_to_merger_table, on=['SEED', 'Metallicity@ZAMS(1)', 'unique_Z_SEED'], how='left')
    print('done with MT_leading_to_merger_table')
    # Merge this info with the potential_DCO_progenitors
    potential_DCO_progenitors = potential_DCO_progenitors.merge(first_MT_from_star2_table, on=['SEED', 'Metallicity@ZAMS(1)', 'unique_Z_SEED'], how='left')
    print('done with first_MT_from_star2_table')

    #################################################################################
    # Save the dataframe
    print('Done!, Saving the potential DCO progenitors with MT info')
    potential_DCO_progenitors.to_hdf(datar_root+ f'/{sim_name}/{save_name_table}', key='All_DCO', mode='w')





Reading DCO seeds from logZ-1.85
Reading DCO seeds from logZ-3.26
Reading DCO seeds from logZ-1.52
Reading DCO seeds from logZ-3.0
Reading DCO seeds from logZ-2.4
Reading DCO seeds from logZ-1.7
Reading DCO seeds from logZ-2.7
Reading DCO seeds from logZ-3.52
Reading DCO seeds from logZ-3.76
Reading DCO seeds from logZ-2.0
Reading DCO seeds from logZ-2.2
Reading DCO seeds from logZ-4.0
start reading RLOF data
add a few extra cols to RLOF
Adding the MT information for the first MT 


/tmp/ipykernel_1403065/356593650.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_RLOF_table.rename(columns={col: 'firstMT_' + col for col in first_RLOF_table.columns if col not in ['SEED', 'Metallicity@ZAMS(1)', 'unique_Z_SEED']}, inplace=True)


this should be 1 [1]
Adding the MT information for the mass transfer that lead to a merger


/tmp/ipykernel_1403065/356593650.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MT_leading_to_merger_table.rename(columns={col: 'MT_lead_to_merger_' + col for col in MT_leading_to_merger_table.columns if col not in ['SEED', 'Metallicity@ZAMS(1)', 'unique_Z_SEED']}, inplace=True)


this should be 1 [1]
Adding the MT information for the first MT from star 2
this should be 1 [1]
start merging tables
done with first_RLOF_table
done with MT_leading_to_merger_table
done with first_MT_from_star2_table
Done!, Saving the potential DCO progenitors with MT info


/tmp/ipykernel_1403065/356593650.py:121: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['MT_Donor_Hist(1)', 'MT_Donor_Hist(2)', 'unique_Z_SEED'], dtype='object')]

  potential_DCO_progenitors.to_hdf(datar_root+ f'/{sim_name}/{save_name_table}', key='All_DCO', mode='w')


In [26]:
save_name_table = 'potential_DCO_progenitors_Allinfo.h5'

# check if your table exists
if os.path.isfile(datar_root+ f'/{sim_name}/{save_name_table}'):
    print('Table already exists, loading it')
    potential_DCO_progenitors = pd.read_hdf(datar_root + f'/{sim_name}/{save_name_table}', key='All_DCO')

else:
    # Read the beginning of the potential DCO progenitors table that you made above
    potential_DCO_progenitors = pd.read_hdf(f'{datar_root}/{sim_name}/potential_DCO_progenitors_RLOFinfo.h5', key='All_DCO')

    # Initialize a list to store all SEEDS that ever become a DCO
    All_DCO_seeds = []
    
    # Loop over all directories starting wiht "logZ"
    for i, dir in enumerate(os.listdir(datar_root+ f'/{sim_name}/')):
        # Get the seeds that ever become a DCO
        if dir.startswith('logZ'):
            print(f"Reading DCO seeds from {dir}") 
            data = h5.File(datar_root+ f'/{sim_name}/{dir}/COMPAS_Output.h5', 'r')
            DCO_seeds = pd.Series(data['BSE_Double_Compact_Objects']['SEED'][()])
            All_DCO_seeds.extend(DCO_seeds)
            
        else:
            continue
        
    
    # take the unique seeds (some SEEDS might make a DCO at multiple metallicities)
    All_DCO_seeds  = np.unique(All_DCO_seeds)

    # Open the HDF5 file for all systems at a given metallicity
    All_data = h5.File(datar_root+ f'/{sim_name}/COMPAS_Output_combinedZ.h5', 'r')
    #################################################################################
    # Finally, add supernove information
    print('Adding the SN information')
    with h5.File(datar_root+f'/{sim_name}/COMPAS_Output_combinedZ.h5', 'r') as All_data:        
        # Read SN info as pandas dataframes
        SNe = pd.DataFrame()
        
        # Select only the SN events for systems that could potentially become a DCO
        SN_mask = np.in1d(All_data['BSE_Supernovae']['SEED'][()], All_DCO_seeds)

        SN_keys_of_interest = ['SEED', 'Metallicity@ZAMS(1)', 'SN_Type(SN)', 'Supernova_State']
        for key in SN_keys_of_interest:
            read_data   = All_data['BSE_Supernovae'][key][()]
            SNe[key]    = read_data[SN_mask]

        #Add unique seed key
        SNe['unique_Z_SEED'] = [f"{seed}_{Z:.5f}" for seed, Z in zip(SNe['SEED'], SNe['Metallicity@ZAMS(1)'])]

    # # Star 1 is going SN
    # star1_SN = SNe[SNe['Supernova_State'] == 1]
    # # Star 2 is going SN
    # star2_SN = SNe[SNe['Supernova_State'] == 2]
    
    # Add the SN info to the potential DCO progenitors
    potential_DCO_progenitors['SN_Type(1)'] = potential_DCO_progenitors['unique_Z_SEED'].map(SNe[SNe['Supernova_State'] == 1].set_index('unique_Z_SEED')['SN_Type(SN)']).fillna(-1)
    potential_DCO_progenitors['SN_Type(2)'] = potential_DCO_progenitors['unique_Z_SEED'].map(SNe[SNe['Supernova_State'] == 2].set_index('unique_Z_SEED')['SN_Type(SN)']).fillna(-1)

    del SNe # empty SNe to save memory
    gc.collect()  # Force garbage collector to release unreferenced memory

    #################################################################################
    # Save the dataframe
    print('Done!, Saving the potential DCO progenitors with SN info')
    potential_DCO_progenitors.to_hdf(datar_root+ f'/{sim_name}/{save_name_table}', key='All_DCO', mode='w')




Reading DCO seeds from logZ-1.85
Reading DCO seeds from logZ-3.26
Reading DCO seeds from logZ-1.52
Reading DCO seeds from logZ-3.0
Reading DCO seeds from logZ-2.4
Reading DCO seeds from logZ-1.7
Reading DCO seeds from logZ-2.7
Reading DCO seeds from logZ-3.52
Reading DCO seeds from logZ-3.76
Reading DCO seeds from logZ-2.0
Reading DCO seeds from logZ-2.2
Reading DCO seeds from logZ-4.0
Adding the SN information
Done!, Saving the potential DCO progenitors with SN info


/tmp/ipykernel_1403065/3203042940.py:66: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['MT_Donor_Hist(1)', 'MT_Donor_Hist(2)', 'unique_Z_SEED'], dtype='object')]

  potential_DCO_progenitors.to_hdf(datar_root+ f'/{sim_name}/{save_name_table}', key='All_DCO', mode='w')


## Finally, split your potential DCO table between potential BBH, BHNS and NSNS

In [27]:
with h5.File(datar_root+f'{sim_name}/COMPAS_Output_combinedZ.h5', 'r') as All_data:
    DCO = All_data['BSE_Double_Compact_Objects']
    st1 = DCO['Stellar_Type(1)'][()]
    st2 = DCO['Stellar_Type(2)'][()]
    dco_merger = DCO['Merges_Hubble_Time'][()]  
    DCO_seed = DCO['SEED'][()]
    # Now I want to add a bool that tells me if this system is ever a BBH, BHNS or BNS progenitor
    BBH_bool = np.logical_and(st1 == 14,st2 == 14)
    BHNS_bool = np.logical_or(np.logical_and(st1 == 13,st2 == 14),
                            np.logical_and(st1 == 14,st2 == 13) )
    NSNS_bool = np.logical_and(st1 == 13,st2 == 13)
    merger_bool = dco_merger == 1

    # Split our potential DCO progenitors into BBH, BHNS and NSNS progenitors
    potential_BBH_progenitors  = potential_DCO_progenitors[np.in1d(potential_DCO_progenitors['SEED'], np.unique(DCO_seed[BBH_bool*merger_bool]) )]
    potential_BHNS_progenitors = potential_DCO_progenitors[np.in1d(potential_DCO_progenitors['SEED'], np.unique(DCO_seed[BHNS_bool*merger_bool]) )]
    potential_NSNS_progenitors = potential_DCO_progenitors[np.in1d(potential_DCO_progenitors['SEED'], np.unique(DCO_seed[NSNS_bool*merger_bool]) )]



In [28]:
print(potential_BBH_progenitors.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707376 entries, 1 to 7954459
Data columns (total 95 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SEED                                  707376 non-null  uint64 
 1   Metallicity@ZAMS(1)                   707376 non-null  float64
 2   Stellar_Type(1)                       707376 non-null  int32  
 3   Stellar_Type(2)                       707376 non-null  int32  
 4   CE_Event_Counter                      707376 non-null  uint32 
 5   Mass@ZAMS(1)                          707376 non-null  float64
 6   Mass@ZAMS(2)                          707376 non-null  float64
 7   SemiMajorAxis@ZAMS                    707376 non-null  float64
 8   Merger                                707376 non-null  uint8  
 9   Merger_At_Birth                       707376 non-null  uint8  
 10  Unbound                               707376 non-null  uint8  
 11 